## Task 1

In [42]:
import pandas as pd

In [44]:
df = pd.read_csv('data.csv', delim_whitespace=True)

/tmp/ipykernel_2826/706671905.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df2 = pd.read_csv('data2.csv', delim_whitespace=True)


In [2]:
# Replace TRUE/FALSE with 1/0 in the 'Windy' column
df['Windy'] = df['Windy'].replace({'TRUE': 1, 'FALSE': 0})

# Convert 'yes'/'no' in 'Play' column to 1/0
df['Play'] = df['Play'].replace({'yes': 1, 'no': 0})

/tmp/ipykernel_2826/3275708656.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Play'] = df['Play'].replace({'yes': 1, 'no': 0})


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def label_encode_columns(df, columns):
    """
    Label encodes the specified columns in a pandas DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame to encode.
    columns (list): A list of column names to label encode.

    Returns:
    pd.DataFrame: DataFrame with label encoded columns.
    """
    le = LabelEncoder()
    
    for col in columns:
        if df[col].dtype == 'object':  # Ensuring it's a categorical column
            df[col] = le.fit_transform(df[col])
    
    return df


In [4]:
encoded_df = label_encode_columns(df, ['Outlook', 'Temperature', 'Humidity'])
encoded_df['Windy'] = df['Windy'].replace({True: 1, False: 0})
# Convert True/False in 'Windy' and other encoded columns to 1/0
df_encoded = encoded_df.astype(int)

/tmp/ipykernel_2826/42272525.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  encoded_df['Windy'] = df['Windy'].replace({True: 1, False: 0})


In [5]:
encoded_df

,Outlook,Temperature,Humidity,Windy,Play
0,0,1,0,0,1
1,0,0,1,1,1
2,0,2,0,1,1
3,0,1,1,0,1
4,1,2,0,0,1
5,1,0,1,0,1
6,1,0,1,1,0
7,1,2,1,0,1
8,1,2,0,1,0
9,2,1,0,0,0


In [15]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% train, 20% test)
train_df, test_df = train_test_split(df_encoded, test_size=0.25, random_state=25)


In [16]:
train_df

,Outlook,Temperature,Humidity,Windy,Play
11,2,2,0,0,0
1,0,0,1,1,1
12,2,0,1,0,1
13,2,2,1,1,1
8,1,2,0,1,0
2,0,2,0,1,1
7,1,2,1,0,1
6,1,0,1,1,0
10,2,1,0,1,0
4,1,2,0,0,1


In [17]:
test_df

,Outlook,Temperature,Humidity,Windy,Play
0,0,1,0,0,1
9,2,1,0,0,0
3,0,1,1,0,1
5,1,0,1,0,1


## Task 2

In [18]:
# Check if the dimension of test_df is equal to or less than train_df
if test_df[0:1].shape[1] == train_df[0:1].shape[1]:
    print(f"Test set dimension {test_df[0:1].shape[1]} is equal to the training set dimension {train_df[0:1].shape[1]}.")
elif test_df[0:1].shape[1] < train_df[0:1].shape[1]:
    print(f"Test set dimension {test_df[0:1].shape[1]} is less than the training set dimension {train_df[0:1].shape[1]}.")


Test set dimension 5 is equal to the training set dimension 5.


In [25]:
import numpy as np

class NaiveBayesClassifier:
    def __init__(self):
        self.prior_probs = {}    # Store prior probabilities for each class
        self.likelihoods = {}    # Store likelihoods for each feature value per class

    def fit(self, X, y):
        # Get unique classes
        self.classes = np.unique(y)

        # Calculate prior probabilities for each class
        print(f"N: number of observations in the train dataset = {len(y)}")
        print(f"n_i: count of values of each class type")
        print("\n")
        for cls in self.classes:
            self.prior_probs[cls] = np.sum(y == cls) / len(y)
            print(f"Prior probability for class {self.classes[cls]}: n_{cls} ({np.sum(y == cls)}) / N ({len(y)}) = {np.sum(y == cls) / len(y)}")
        print("\n")
        # Calculate likelihoods for each feature given the class
        print(f"N: number of feature values given a specific class")
        print(f"n_i: count of values of each feature type given a class")
        print("\n")
        for cls in self.classes:
            self.likelihoods[cls] = {}
            class_data = X[y == cls]  # Subset of data where y == cls
            print(f"Let's calculate likelihoods for all features given class {self.classes[cls]}\n")
            for feature in X.columns:
                self.likelihoods[cls][feature] = {}
                print(f"feature {feature}")
                feature_vals = X[feature].unique()  # Unique values for the feature
                print(f"for this feature, we have {len(feature_vals)} types.")
                
                for val in feature_vals:
                    # P(feature | class) = (count of feature value in class) / (total count of class)
                    count = np.sum(class_data[feature] == val)
                    self.likelihoods[cls][feature][val] = count / len(class_data)
                    print(f"P(feature={val} | class={cls} ) = count of a feature type ({count}) / total count of class ({len(class_data)}) = {count / len(class_data)}")
                print(f"\n")
        print("\n")
    def predict(self, X_test):
        predictions = []
        for _, row in X_test.iterrows():
            # For each test instance, calculate P(class | features) for each class
            prior_probs = {}

            for cls in self.classes:
                # Start with prior probability
                prior_probs[cls] = self.prior_probs[cls]

                # Multiply by likelihoods for each feature
                for feature in X_test.columns:
                    feature_val = row[feature]
                    if feature_val in self.likelihoods[cls][feature]:
                        prior_probs[cls] *= self.likelihoods[cls][feature][feature_val]
                    else:
                        # If a feature value wasn't observed, multiply by a small value to avoid zero probability
                        prior_probs[cls] *= 1e-6

            # Choose class with highest probability
            predicted_class = max(prior_probs, key=prior_probs.get)
            predictions.append(predicted_class)

        return np.array(predictions)



In [26]:
# Example Usage
# Assuming df_encoded is already split into train_df and test_df, and 'Play' is the target column.

# Features (X) and Target (y)
X_train = train_df.drop('Play', axis=1)
y_train = train_df['Play']

X_test = test_df.drop('Play', axis=1)
y_test = test_df['Play']

# Train Naive Bayes Classifier
nb_classifier = NaiveBayesClassifier()
nb_classifier.fit(X_train, y_train)

# Predict on test set
y_pred = nb_classifier.predict(X_test)

# Display predictions
print("Predicted labels:", y_pred)
print("Actual labels:", y_test.values)

N: number of observations in the train dataset = 10
n_i: count of values of each class type


Prior probability for class 0: n_0 (4) / N (10) = 0.4
Prior probability for class 1: n_1 (6) / N (10) = 0.6
Prior probability list for all classes: {np.int64(0): np.float64(0.4), np.int64(1): np.float64(0.6)}


N: number of feature values given a specific class
n_i: count of values of each feature type given a class


Let's calculate likelihoods for all features given class 0

feature Outlook
for this feature, we have 3 types.
P(feature=2 | class=0 ) = count of a feature type (2) / total count of class (4) = 0.5
P(feature=0 | class=0 ) = count of a feature type (0) / total count of class (4) = 0.0
P(feature=1 | class=0 ) = count of a feature type (2) / total count of class (4) = 0.5


feature Temperature
for this feature, we have 3 types.
P(feature=2 | class=0 ) = count of a feature type (2) / total count of class (4) = 0.5
P(feature=0 | class=0 ) = count of a feature type (1) / total count of

## Task 3

In [36]:
import numpy as np

class NaiveBayesClassifierWithLaplace:
    def __init__(self, alpha=1):
        """
        Naive Bayes Classifier with Laplace Smoothing.

        Parameters:
        smoothing (float): The Laplace smoothing parameter 'a'.
        """
        self.alpha = alpha
        self.prior_probs = {}  # Prior probabilities P(C)
        self.likelihoods = {}  # Likelihood probabilities P(x|C)

    def fit(self, X, y):
        """
        Fit the Naive Bayes model on the training data.

        Parameters:
        X (pd.DataFrame): Training features.
        y (pd.Series): Training labels.
        """
        self.classes = np.unique(y)
        total_samples = len(y)
        self.class_counts = {}

        # Calculate prior probabilities P(C)
        print(f"N: number of observations in the train dataset = {total_samples}")
        print(f"n_i: count of values of each class type")
        print(f"a: smoothing parameter = {self.alpha}")
        print(f"v: count of possible unique values in classes = {len(self.classes)}")
        print("\n")
        for cls in self.classes:
            class_count = sum(y == cls)
            self.class_counts[cls] = class_count  # Store class count
            self.prior_probs[cls] = (class_count + self.alpha) / (total_samples + len(self.classes) * self.alpha)
            print(f"Prior probability for class {self.classes[cls]} = n_{cls} + a / N + a x v: {(class_count + self.alpha) / (total_samples + len(self.classes) * self.alpha)}")
        print("\n")
        
        # Calculate likelihood probabilities P(x|C) with Laplace smoothing
        print(f"N: number of feature values given a specific class")
        print(f"n_i: count of values of each feature type given a class")
        print(f"a: smoothing parameter = {self.alpha}")
        print(f"v: count of possible unique values in each feature")
        print("\n")
        for cls in self.classes:
            self.likelihoods[cls] = {}
            class_data = X[y == cls]  # Subset of data where y == cls
            print(f"Let's calculate likelihoods for all features given class {self.classes[cls]}\n")
            
            for feature in X.columns:
                self.likelihoods[cls][feature] = {}
                print(f"feature {feature}")
                feature_vals = X[feature].unique()  # Unique values for the feature
                v = len(feature_vals)
                class_feature_values = class_data[feature]
                N = len(class_feature_values)
                print(f"for this feature, we have {len(feature_vals)} types.")
                
                for val in feature_vals:
                    # P(feature | class) = (count of feature value in class) / (total count of class)
                    n_i = np.sum(class_data[feature] == val)
                    self.likelihoods[cls][feature][val] = (n_i + self.alpha) / (N + v * self.alpha)
                    print(f"P(feature {val} | class {cls} ) = count of a feature ({n_i}) x a ({self.alpha})/ N ({N}) = {(n_i + self.alpha) / (N + v * self.alpha)}")
                print(f"")
    
    
    def predict(self, X):
        """
        Predict the class labels for the provided data.

        Parameters:
        X (pd.DataFrame): Data to predict.

        Returns:
        np.array: Predicted class labels.
        """
        predictions = []

        for _, row in X.iterrows():
            class_scores = {}

            for cls in self.classes:
                # Start with the log of the prior probability
                class_scores[cls] = np.log(self.prior_probs[cls])

                # Add the log likelihoods P(x|C) for each feature
                for feature in X.columns:
                    feature_value = row[feature]

                    if feature_value in self.likelihoods[cls][feature]:  # Corrected access
                        class_scores[cls] += np.log(self.likelihoods[cls][feature][feature_value])
                    else:
                        # If the value hasn't been observed, apply Laplace smoothing
                        unique_values = len(self.likelihoods[cls][feature])
                        class_scores[cls] += np.log(self.alpha / (self.class_counts[cls]) + unique_values * self.alpha)

            # Predict the class with the highest score
            predictions.append(max(class_scores, key=class_scores.get))

        return np.array(predictions)





In [35]:
# Example usage
if __name__ == "__main__":
    
    # Features (X) and Target (y)
    X_train = train_df.drop('Play', axis=1)
    y_train = train_df['Play']

    X_test = test_df.drop('Play', axis=1)
    y_test = test_df['Play']
    
    # Instantiate and fit the classifier
    clf = NaiveBayesClassifierWithLaplace(alpha=1)  # Laplace smoothing with a=1
    clf.fit(X_train, y_train)

    # Predict
    predictions = clf.predict(X_test)
    print("Predictions:", predictions)
    print("True Values:", y_test)

N: number of observations in the train dataset = 10
n_i: count of values of each class type
a: smoothing parameter = 1
v: count of possible unique values in classes = 2


Prior probability for class 0 = n_0 + a / N + a x v: 0.4166666666666667
Prior probability for class 1 = n_1 + a / N + a x v: 0.5833333333333334


N: number of feature values given a specific class
n_i: count of values of each feature type given a class
a: smoothing parameter = 1
v: count of possible unique values in each feature


Let's calculate likelihoods for all features given class 0

feature Outlook
for this feature, we have f3 types.
P(feature 2 | class 0 ) = count of a feature (2) x a (1)/ N (4) = 0.42857142857142855
P(feature 0 | class 0 ) = count of a feature (0) x a (1)/ N (4) = 0.14285714285714285
P(feature 1 | class 0 ) = count of a feature (2) x a (1)/ N (4) = 0.42857142857142855

feature Temperature
for this feature, we have f3 types.
P(feature 2 | class 0 ) = count of a feature (2) x a (1)/ N (4) = 0.4